# **Início**

In [52]:
import pandas as pd
import collections
import heapq
import time
import operator
import csv
import nltk
import string
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

data = pd.read_csv('./results.csv', sep=',')
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,subtitle,author,date,section,text,url
0,“A sociedade foi Rubens Paiva não os facínora...,A decisão da juíza que proíbe as Forças Armada...,F. M.,30/03/2019 00:11:08,Brasil,A juíza federal Ivani Silva da Luz de Brasíli...,https://brasil.elpais.com/brasil/2019/03/26/po...
1,Justiça suspende decisão que proibia Forças Ar...,Liminar havia sido concedida na sexta-feira a ...,Marina Rossi,30/03/2019 16:17:59,Brasil,Menos de 24 horas depois de a juíza federal Iv...,https://brasil.elpais.com/brasil/2019/03/30/po...
2,Governo Bolsonaro prega “negacionismo históric...,Marcos Napolitano professor da USP diz que o...,Regiane Oliveira,04/04/2019 22:37:48,Brasil,Quando determinou que de 31 de março 1964 u...,https://brasil.elpais.com/brasil/2019/04/05/po...
3,Quando os pais de Gabo perceberam que tinham u...,Gustavo Tatis percorre o universo de García Má...,Jesús Ruiz Mantilla,07/03/2019 16:38:56,Cultura,Quando era pequeno Luisa e Gabriel se preo...,https://brasil.elpais.com/brasil/2019/03/06/cu...
4,Rádios canadenses banem músicas de Michael Jac...,Quebec Cogeco Media toma a decisão após queixa...,Jaime Porras Ferreyra,07/03/2019 16:12:37,Cultura,Desde a manhã da última segunda-feira e ...,https://brasil.elpais.com/brasil/2019/03/06/cu...


# Questão 1

Texto original/Tokenizado :

In [53]:
data = pd.DataFrame(data['text'], columns=['text'])
data['tokenized_text'] = data.apply(lambda row: nltk.word_tokenize(row['text'].lower(), language='portuguese'), axis=1)
data.head()

,text,tokenized_text
0,A juíza federal Ivani Silva da Luz de Brasíli...,"[a, juíza, federal, ivani, silva, da, luz, de,..."
1,Menos de 24 horas depois de a juíza federal Iv...,"[menos, de, 24, horas, depois, de, a, juíza, f..."
2,Quando determinou que de 31 de março 1964 u...,"[quando, determinou, que, de, 31, de, março, 1..."
3,Quando era pequeno Luisa e Gabriel se preo...,"[quando, era, pequeno, luisa, e, gabriel, se, ..."
4,Desde a manhã da última segunda-feira e ...,"[desde, a, manhã, da, última, segunda-feira, e..."


Construindo o csv com índice invertido:

In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')
numberOfDocuments = 0
index = {}

for wordList in data.tokenized_text:
  numberOfDocuments += 1
  for word in wordList:
    if word not in stopwords and len(word) >= 3:      
      if word not in index.keys():
        index[word] = []
      index[word].append(numberOfDocuments)
      
for elem in index.items():
  dic = dict(collections.Counter(elem[1]))
  index[elem[0]] = list(dic.items())  
  
  
indexed_data = pd.DataFrame()

indexed_data['Word'] = [word for word in index.keys()]
indexed_data['Documents'] = [docs for docs in index.values()]

indexed_data.to_csv('indexed.csv', encoding='utf-8', index=False)

# Questão 2

Implementação das formas de consulta ao índice:

-Documento por vez

-Termo por vez


In [55]:
import time

def documentAtTime(query, index, k):
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_pos in range(1, len(data) + 1):
        sd = 0
        for document_count in document_count_list:
            for i in document_count:
                if (i[0] == document_pos):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, document_pos))
    
    return heapq.nlargest(k, r)
  
def termAtTime(query, index, k):
    a = {}
    document_count_list = []
    r = []
    for word in query.split():
        document_count_list.append(index[word])
    for document_count in document_count_list:
        for post in document_count:
            d = post[0]
            freq = post[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))

    return heapq.nlargest(k, r)

  
queries = ["presidente", "ministério", "bolsonaro", "país", "economia"]
k = 10
resultsDoc = []
resultsTerm = []

for query in queries:
  scoreDoc = documentAtTime(query, index, k)
  resultsDoc.append(scoreDoc)
  scoreTerm = termAtTime(query, index, k)
  resultsTerm.append(scoreTerm)

queries_df = pd.DataFrame()
queries_df['Consulta'] = queries
queries_df['Documento por vez'] = resultsDoc
queries_df['Termo por vez'] = resultsTerm
queries_df['Mesmo resultado'] = queries_df['Documento por vez'] == queries_df['Documento por vez']
queries_df
        
def document1000TimesPerQuery():
  for query in queries:
    for _ in range(10000):
      documentAtTime(query, index, k)

def term1000TimesPerQuery():
  for query in queries:
    for _ in range(1000):
      termAtTime(query, index, k)
      
qtdQueriesExec = 1000 * 5

documentStart = time.time()
document1000TimesPerQuery()
documentTimeElapsed = time.time() - documentStart
documentAvgTime = documentTimeElapsed / qtdQueriesExec
print("Tempo médio de 'Documento Por Vez': ", documentAvgTime)

termStart = time.time()
term1000TimesPerQuery()
termTimeElapsed = time.time() - termStart
termAverageTime = termTimeElapsed / qtdQueriesExec
print("Tempo de 'Termo Por Vez': ", termAverageTime)

Tempo médio de 'Documento Por Vez':  0.012011565446853638
Tempo de 'Termo Por Vez':  8.739895820617676e-05


# **Questão 3**

In [56]:
def conjuctiveQuery(query, indexes, k):
  query_indexes = []
  rank = []
  
  for word in query.split(" "):
    if word in indexes.keys():
        query_indexes.append(indexes[word])
  
  all_indexes = [item for sublist in query_indexes for item in sublist]
  all_indexes.sort()
  
  for i in range(len(all_indexes)):
    doc_score = 0
    d = all_indexes.pop()
    repeat = 1
    for index in all_indexes:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

queries2 = ["direitos humanos", "lula livre", "política ruptura", "corte interamericana", "país política"]

k = 10
conjResults = []

for query in queries2:
  conjuctiveScore = conjuctiveQuery(query, index, k)
  conjResults.append(conjuctiveScore)

conj_queries_df = pd.DataFrame()
conj_queries_df['Query'] = queries2
conj_queries_df['Resultados'] = conjResults
conj_queries_df

,Query,Resultados
0,direitos humanos,"[(7, 117), (6, 153), (6, 1), (4, 166), (3, 173..."
1,lula livre,"[(2, 238), (2, 172), (2, 26)]"
2,política ruptura,"[(8, 166), (6, 152), (2, 83), (2, 25), (2, 1)]"
3,corte interamericana,"[(3, 223), (3, 1)]"
4,país política,"[(18, 166), (16, 173), (12, 69), (11, 138), (9..."
